In [ ]:
#importing the libraries needed 
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from time import sleep
from random import randint

In [ ]:
#Declaring the headers 
headers = {"Accept-Language": "en-US,en;q=0.5"}


In [ ]:
#creating an empty list, so that we can append the values
movie_name = []
genres = []
year = []
time = []
rating = []
metascore = []
votes = []


In [ ]:
#creating an array of values and passing it in the url for dynamic webpages
pages = np.arange(1,1000,100)

In [ ]:
for page in pages:
    page = requests.get("https://www.imdb.com/search/title/?groups=top_1000&sort=user_rating,desc&count=100&start="+str(page)+"&ref_=adv_nxt")
    soup = BeautifulSoup(page.text, 'html.parser')
    movie_data = soup.findAll('div', attrs = {'class': 'lister-item mode-advanced'})
    sleep(randint(2,8))
    #calling one by one using for loop
    for store in movie_data:
        name = store.h3.a.text
        movie_name.append(name)
        
        genre = store.p.find('span', class_ = 'genre').text.replace('\n', '')
        genres.append(genre)

        year_of_release = store.h3.find('span', class_ = 'lister-item-year text-muted unbold').text.replace('(', '').replace(')', '')
        year.append(year_of_release)
        
        runtime = store.p.find('span', class_ = 'runtime').text.replace(' min', '')
        time.append(runtime)
        
        rate = store.find('div', class_ = 'inline-block ratings-imdb-rating').text.replace('\n', '')
        rating.append(rate)
        
        meta  = store.find('span', class_ = 'metascore').text.replace(' ', '') if store.find('span', class_ = 'metascore') else '^^^^^^'
        metascore.append(meta)

        value = store.find_all('span', attrs = {'name': 'nv'})
        vote = value[0].text
        votes.append(vote)


In [ ]:
#creating a dataframe using pandas library
movie_DF = pd.DataFrame({'Name of movie': movie_name, 'Genres':genres, 'Year of release': year, 'Watch time': time, 'Movie Rating': rating, 'Meta score': metascore, 'Votes':votes})
movie_DF

,Name of movie,Genres,Year of release,Watch time,Movie Rating,Meta score,Votes
0,刺激1995,Drama,1994,142,9.3,81,"2,651,602"
1,教父,"Crime, Drama",1972,175,9.2,100,"1,838,082"
2,黑暗騎士,"Action, Crime, Drama",2008,152,9.0,84,"2,623,775"
3,魔戒三部曲：王者再臨,"Action, Adventure, Drama",2003,201,9.0,94,"1,827,506"
4,辛德勒的名單,"Biography, Drama, History",1993,195,9.0,94,"1,343,676"
...,...,...,...,...,...,...,...
995,龍鳳配,"Comedy, Drama, Romance",1954,113,7.6,72,"65,451"
996,亂世忠魂,"Drama, Romance, War",1953,118,7.6,85,"47,307"
997,白雪公主,"Animation, Adventure, Family",1937,83,7.6,95,"199,572"
998,國防大機密,"Crime, Mystery, Thriller",1935,86,7.6,93,"57,361"


In [ ]:
movie_DF.to_csv("Top_1000_IMDB_Movies.csv")

In [ ]:
pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 38 kB/s 
     |████████████████████████████████| 199 kB 44.4 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=b84a771adf185b4c2090c4577f52b30bcf835852e28777afdb8e74fe5f1ab3d8
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark


sử dụng PySpark để load dữ liệu 
từ file csv thu thập được. 

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
spark = SparkSession.builder.appName("BT2_HoThiMinhNguyen").getOrCreate()
movie_DF = spark.read.csv('Top_1000_IMDB_Movies.csv',header=True,inferSchema=True)
MOVIE_DF = movie_DF
MOVIE_DF.show(10)
MOVIE_DF.printSchema()

+---+--------------------+--------------------+---------------+----------+------------+----------+---------+
|_c0|       Name of movie|              Genres|Year of release|Watch time|Movie Rating|Meta score|    Votes|
+---+--------------------+--------------------+---------------+----------+------------+----------+---------+
|  0|            刺激1995|   Drama            |           1994|       142|         9.3|        81|2,651,602|
|  1|                教父|Crime, Drama     ...|           1972|       175|         9.2|       100|1,838,082|
|  2|            黑暗騎士|Action, Crime, Dr...|           2008|       152|         9.0|        84|2,623,775|
|  3|魔戒三部曲：王者再臨|Action, Adventure...|           2003|       201|         9.0|        94|1,827,506|
|  4|        辛德勒的名單|Biography, Drama,...|           1993|       195|         9.0|        94|1,343,676|
|  5|               教父2|Crime, Drama     ...|           1974|       202|         9.0|        90|1,259,487|
|  6|            十二怒漢|Crime, Drama     ...|  